In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from google.colab import files

In [2]:
#importing data
upload = files.upload()
predata = pd.read_csv('data.csv')
predata = predata[['Year', 'Value']]
predata = predata.rename(columns={'Year':'Year', 'Value':'Value'})

#grouping data
data = predata.groupby('Year')['Value'].mean().reset_index()
list_rmse = []

#splitting and preparing data
train = data[data['Year'] < 2011]
over_2011 = train
test = data[data['Year'] >= 2011]
scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

Saving data.csv to data.csv


In [5]:
#list of all predicted values from all 50 runs + variables for avg predicted values per year
total_pred = []

#running the programme 50 times
for i in range(50):
  print(f"Run number:{i}")
  x_train = train[:, :-1]
  y_train = train[:, -1]
  x_test = test[:, :-1]
  y_test = test[:, -1]

  x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
  x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
#LSTM layers
  model = Sequential()
  model.add(LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
  model.add(LSTM(100, return_sequences=True))
  model.add(LSTM(100, return_sequences=False))
  model.add(Dense(1))
#compiling
  model.compile(optimizer='adam', loss='mean_squared_error')
  history = model.fit(x_train, y_train, batch_size=24, epochs=50, verbose=0)
#storing the predictions
  y_pred = model.predict(x_test)

#saving the predictions
  #for i in y_pred:
    #total_pred.append(y_pred[i])

  y_test = np.hstack((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], 1))))
  y_pred = np.hstack((y_pred.reshape(-1, 1), np.zeros((y_pred.shape[0], 1))))
  y_test_inv = scaler.inverse_transform(y_test)[:, 0]
  y_pred_inv = scaler.inverse_transform(y_pred)[:, 0]

  test_df = pd.DataFrame(test, columns=['Year', 'Value'])
  test_df['Value'] = y_test
  y_test_inverse = scaler.inverse_transform(test_df)[:, -1]

  pred_df = pd.DataFrame(test, columns=['Year', 'Value'])
  pred_df['Value'] = y_pred
  y_pred_inverse = scaler.inverse_transform(pred_df)[:, -1]

  total_pred.extend(y_pred_inverse.flatten())

  rmse = np.sqrt(np.mean((y_pred_inverse - y_test_inverse)**2))
  list_rmse.append(rmse)

Run number:0
1/1 [==============================] - 1s 846ms/step
Run number:1
1/1 [==============================] - 1s 819ms/step
Run number:2


1/1 [==============================] - 1s 1s/step
Run number:3


1/1 [==============================] - 1s 1s/step
Run number:4
1/1 [==============================] - 1s 1s/step
Run number:5
1/1 [==============================] - 1s 848ms/step
Run number:6
1/1 [==============================] - 1s 867ms/step
Run number:7
1/1 [==============================] - 1s 1s/step
Run number:8
1/1 [==============================] - 1s 857ms/step
Run number:9
1/1 [==============================] - 1s 850ms/step
Run number:10
1/1 [==============================] - 1s 866ms/step
Run number:11
1/1 [==============================] - 1s 833ms/step
Run number:12
1/1 [==============================] - 1s 986ms/step
Run number:13
1/1 [==============================] - 1s 880ms/step
Run number:14
1/1 [==============================] - 1s 879ms/step
Run number:15
1/1 [==============================] - 1s 855ms/step
Run number:16
1/1 [==============================] - 1s 851ms/step
Run number:17
1/1 [==============================] - 1s 906ms/step
Run number:18
1/1 [=====

In [6]:
#calculating avg. rmse value from all 50 runs
avg_rmse = sum(list_rmse)/len(list_rmse)
print('Average RMSE: %.2f' % avg_rmse)

Average RMSE: 0.73


In [8]:
#CREATING A LIST OF AVERAGE PREDICTIONS PER YEAR OVER 50 RUNS
#for i in range(13):
 #   cont = []
  #  avg_pred=[]
   # for j in range(i, len(total_pred), 13):
    #    cont.append(total_pred[j])
     #   avg_prediction = sum(cont) / len(cont)
    #fin_avg_pred.append(avg_prediction)
#(f"Average Predictions: {fin_avg_pred}")